# https://github.com/unit8co/darts/issues/1814

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

data = pd.DataFrame(
    {"a": [1] * 24 * 7 * 5, "b": [2] * 24 * 7 * 5},
    index=pd.date_range(start="2023-01-01", freq="1H", periods=24 * 7 * 5),
)

from darts import TimeSeries
from darts.models import RandomForest, RegressionModel
from darts.models.forecasting.regression_ensemble_model import RegressionEnsembleModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import r2_score, mape

TARGET = "a"
COVARIATES = ["b"]

data = TimeSeries.from_dataframe(data)

data = data.add_datetime_attribute("hour", one_hot=True)

COVARIATES = COVARIATES + [f"hour_{i}" for i in range(1, 25)]


def _run():
    model1 = RandomForest(
        lags_future_covariates=[0],
        n_estimators=500,
        add_encoders={
            "datetime_attribute": {"future": ["hour"]},
            "transformer": Scaler(),
            "position": {"future": "relative"},
        },
    )
    print("forest extreme lags: ", model1.extreme_lags)
    model2 = RegressionModel(
        lags_future_covariates=[0],
        add_encoders={
            # "datetime_attribute": {"future": ["hour"]},
            "transformer": Scaler(),
            "position": {"future": "relative"},
        },
    )
    print("linear extreme lags: ", model2.extreme_lags)

    class M(RegressionEnsembleModel):
        """so monkey it's sexy"""

        @property
        def extreme_lags(
            self,
        ):
            return (None, 0, None, None, 0, 0)

    model = M(
        forecasting_models=[model1, model2],
        regression_train_n_points=50,
        regression_model=RandomForestRegressor(),
    )

    print("ensemble extreme lags: ", model.extreme_lags)
    future_covariates = data[[x for x in data.columns if x in COVARIATES]]

    results = model.historical_forecasts(
        series=data[TARGET],
        future_covariates=future_covariates,
        forecast_horizon=1 * 24,
        stride=1 * 24,
        start=24 * 7 * 4,
        train_length=24 * 7 * 4,
        last_points_only=False,
    )

    return results


d = _run()

/home/dev/miniconda3/envs/pf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


forest extreme lags:  (None, 0, None, None, 0, 0)
linear extreme lags:  (None, 0, None, None, 0, 0)
ensemble extreme lags:  (None, 0, None, None, 0, 0)


# 资料

* https://github.com/h3ik0th/Darts_ensembleFC/blob/main/darts16_update2022.01.ipynb
